In [106]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [107]:
import sys
sys.path.append("..")

In [108]:
from optimus import Optimus

In [111]:
# !pip install streamz

In [112]:
from streamz import Stream

def query(df):
    return df[df.name == 'Alice']

def aggregate(acc, df):
    return acc + df.amount.sum()

stream = Stream()
stream.map(query).accumulate(aggregate, start=0)

Output()

In [133]:
import pandas as pd
from streamz.dataframe import DataFrame

example = pd.DataFrame({'name': [], 'amount': []})
example = pd.read_csv("data/crime.csv", encoding ="latin-1")

In [134]:
example.dtypes

INCIDENT_NUMBER         object
OFFENSE_CODE             int64
OFFENSE_CODE_GROUP      object
OFFENSE_DESCRIPTION     object
DISTRICT                object
REPORTING_AREA          object
SHOOTING                object
OCCURRED_ON_DATE        object
YEAR                     int64
MONTH                    int64
DAY_OF_WEEK             object
HOUR                     int64
UCR_PART                object
STREET                  object
Lat                    float64
Long                   float64
Location                object
dtype: object

In [139]:
from streamz.dataframe import Aggregation

class Mean(Aggregation):
    def initial(self, new):
        state = new.iloc[:0].sum(), new.iloc[:0].count()
        return state

    def on_new(self, state, new):
        total, count = state
        total = total + new.sum()
        count = count + new.count()
        new_state = (total, count)
        new_value = total / count
        return new_state, new_value

    def on_old(self, state, old):
        total, count = state
        total = total - old.sum()   # switch + for - here
        count = count - old.count() # switch + for - here
        new_state = (total, count)
        new_value = total / count
        return new_state, new_value

In [141]:
# https://streamz.readthedocs.io/en/latest/dataframes.html
sdf = DataFrame(stream, example=example)
sdf["OFFENSE_CODE"].aggregate(Mean())

Output()

In [31]:
example.dtypes

INCIDENT_NUMBER         object
OFFENSE_CODE             int64
OFFENSE_CODE_GROUP      object
OFFENSE_DESCRIPTION     object
DISTRICT                object
REPORTING_AREA          object
SHOOTING                object
OCCURRED_ON_DATE        object
YEAR                     int64
MONTH                    int64
DAY_OF_WEEK             object
HOUR                     int64
UCR_PART                object
STREET                  object
Lat                    float64
Long                   float64
Location                object
dtype: object

In [42]:
# source = Stream.filenames('data/crime.csv')  # stream of filenames
# sdf = (source.map(pd.read_csv)                  # stream of Pandas dataframes
#              .map(lambda df: df.value.sum()))        # logical streaming dataframe
import pandas as pd
from streamz.dataframe import DataFrame
from streamz import Stream

stream = Stream()
df =pd.read_csv('data/crime.csv', encoding="latin1")
sdf = DataFrame(stream, example=df)

# https://amplitude.com/blog/2014/08/06/optimal-streaming-histograms

def query(df):
    print(type(df))
    return df[df.name == 'Alice']

def aggregate(acc, df):
    return acc + df.amount.sum()

# stream.map(query).accumulate(aggregate, start=0)
# stream.accumulate(aggregate, start=0)
print(sdf.window(10))

# print(sdf["OFFENSE_CODE"])
# print(sdf[sdf["DAY_OF_WEEK"] == 'Sunday']["OFFENSE_CODE"].sum())
# sdf.mean().stream.sink(print)                   # printed stream of mean values
# source.emit(sdf)

In [162]:
example.dtypes

INCIDENT_NUMBER         object
OFFENSE_CODE             int64
OFFENSE_CODE_GROUP      object
OFFENSE_DESCRIPTION     object
DISTRICT                object
REPORTING_AREA          object
SHOOTING                object
OCCURRED_ON_DATE        object
YEAR                     int64
MONTH                    int64
DAY_OF_WEEK             object
HOUR                     int64
UCR_PART                object
STREET                  object
Lat                    float64
Long                   float64
Location                object
dtype: object

In [172]:
from streamz.dataframe import Random, DataFrame

,x,y,z
2020-04-04 17:38:19.386549312,29.700659,1545.5,-1679.705154
2020-04-04 17:38:19.391549312,29.358314,1545.0,-1680.776638
2020-04-04 17:38:19.396549312,29.603302,1547.5,-1681.250546
2020-04-04 17:38:19.401549312,29.712938,1550.0,-1682.958273
2020-04-04 17:38:19.406549312,29.835774,1552.5,-1683.391827


In [173]:
source = Random(freq='5ms', interval='100ms')
source.x.sum()

Output()

In [188]:
sdf = (source - 0.5).cumsum()
sdf.tail()

,x,y,z
2020-04-04 17:47:35.206990144,-3.337679,146.0,-160.032365
2020-04-04 17:47:35.211990144,-3.466908,146.5,-160.505904
2020-04-04 17:47:35.216990144,-3.441963,146.0,-161.248841
2020-04-04 17:47:35.221990144,-3.663467,148.5,-162.549345
2020-04-04 17:47:35.226990144,-3.749223,148.0,-161.985967


In [179]:
p = (DataFrame({'raw': sdf.x,
                        'smooth': sdf.x.rolling('100ms').mean(),
                        'very-smooth': sdf.x.rolling('500ms').mean()})
     .plot(width=700)
    )

3434.293768959762

In [28]:
from optimus import Optimus
op = Optimus("dask", n_workers=4, threads_per_worker=2, processes=False, memory_limit="3G", comm=True)

C:\Users\argenisleon\Anaconda3\lib\site-packages\distributed\bokeh\core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [29]:
# from dask.distributed import Client, LocalCluster
# cluster = LocalCluster()
# client = Client(cluster)

In [30]:
# df.min().compute()

In [31]:
df = op.load.csv("data/foo.csv", sep=",", error_bad_lines=False, header=True, null_value="null", infer_schema='true', charset="latin1").ext.repartition(4).ext.cache()
# .ext.optimize().ext.cache()

In [32]:
df.ext.display("all")

id 1 (int64) not nullable,firstName 2 (object) not nullable,lastName 3 (object) not nullable,billingId 4 (int64) not nullable,product 5 (object) not nullable,price 6 (int64) not nullable,birth 7 (object) not nullable,dummyCol 8 (object) not nullable
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never
2,AndrÃ©,AmpÃ¨re,423,piza,8,1950/07/08,gonna
3,NiELS,BÃ¶hr//((%%,551,pizza,8,1990/07/09,give
4,PAUL,dirac$,521,pizza,8,1954/07/10,you
5,Albert,Einstein,634,pizza,8,1990/07/11,up
6,Galileo,⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅GALiLEI,672,arepa,5,1930/08/12,never
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down


In [101]:
df.ext.set_buffer("*")
pdf = df.ext.get_buffer()

In [105]:
pdf.cols.bucketizer(["id","billingId"],5,"aid")

AAA ['id', 'billingId']
BBB aid


,id,firstName,lastName,billingId,product,price,birth,dummyCol
0,1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never
1,2,AndrÃ©,AmpÃ¨re,423,piza,8,1950/07/08,gonna
2,3,NiELS,BÃ¶hr//((%%,551,pizza,8,1990/07/09,give
3,4,PAUL,dirac$,521,pizza,8,1954/07/10,you


In [79]:
pdf

,id,firstName,lastName,billingId,product,price,birth,dummyCol,aid
0,1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never,0.0
1,2,AndrÃ©,AmpÃ¨re,423,piza,8,1950/07/08,gonna,1.0
2,3,NiELS,BÃ¶hr//((%%,551,pizza,8,1990/07/09,give,3.0
3,4,PAUL,dirac$,521,pizza,8,1954/07/10,you,4.0


In [57]:
pdf.cols.is_match("price","int")

<built-in function isint>


0    True
1    True
2    True
3    True
Name: price, dtype: bool